In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model

2025-02-18 12:51:35.872058: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-18 12:51:35.873229: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-18 12:51:35.895004: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-18 12:51:35.895735: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-18 12:51:36.295318: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

## Encoder Class

In [2]:
class Encoder(Model):
    def __init__(self, latent_dim):
        super(Encoder, self).__init__()
        # Define the layers for the encoder
        self.flatten = layers.Flatten()  # Flatten the image input
        self.dense1 = layers.Dense(512, activation='relu')  # First dense layer
        self.dense2 = layers.Dense(256, activation='relu')  # Second dense layer
        self.z_mean = layers.Dense(latent_dim)  # Mean of the latent distribution
        self.z_log_var = layers.Dense(latent_dim)  # Log variance of the latent distribution

    def call(self, inputs):
        x, label = inputs  # Inputs: (image, label)
        label = tf.one_hot(label, depth=10)  # One-hot encode the labels
        x = tf.concat([x, label], axis=-1)  # Concatenate label with the image
        x = self.flatten(x)  # Flatten the image into a 1D vector
        x = self.dense1(x)  # Pass through the first dense layer
        x = self.dense2(x)  # Pass through the second dense layer
        z_mean = self.z_mean(x)  # Mean of the latent space
        z_log_var = self.z_log_var(x)  # Log variance of the latent space
        return z_mean, z_log_var

# Reparameterization trick (sampling from the latent space)
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.random.normal(tf.shape(z_mean))  # Sample from a normal distribution
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon  # Reparameterization trick

## Decoder class

In [3]:
class Decoder(Model):
    def __init__(self, latent_dim):
        super(Decoder, self).__init__()
        # Define the layers for the decoder
        self.dense1 = layers.Dense(256, activation='relu')  # First dense layer
        self.dense2 = layers.Dense(512, activation='relu')  # Second dense layer
        self.output_layer = layers.Dense(28 * 28, activation='sigmoid')  # Output layer to reconstruct the image

    def call(self, z):
        x = self.dense1(z)  # Pass through the first dense layer
        x = self.dense2(x)  # Pass through the second dense layer
        x = self.output_layer(x)  # Reconstruct the image
        return tf.reshape(x, (-1, 28, 28, 1))  # Reshape back to image shape

## CVAE class

In [4]:
class CVAE(Model):
    def __init__(self, latent_dim):
        super(CVAE, self).__init__()
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim)

    def call(self, inputs):
        x, label = inputs  # Inputs: (image, label)
        z_mean, z_log_var = self.encoder([x, label])  # Get mean and log variance from the encoder
        z = sampling([z_mean, z_log_var])  # Sample from the latent space using the reparameterization trick
        reconstructed = self.decoder(z)  # Reconstruct the image using the decoder
        return reconstructed, z_mean, z_log_var

# Loss function: Reconstruction loss + KL divergence
def compute_loss(x, reconstructed, z_mean, z_log_var):
    # Binary cross-entropy loss for image reconstruction
    cross_entropy = tf.reduce_mean(tf.reduce_sum(
        tf.keras.losses.binary_crossentropy(x, reconstructed), axis=[1, 2, 3]
    ))
    
    # KL divergence between the learned distribution and the prior
    kl_divergence = -0.5 * tf.reduce_mean(
        tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
    )
    
    return cross_entropy + kl_divergence

---